In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!cd /content/drive/MyDrive/
!git clone https://github.com/phuonganhcorn/XLM_Finetune_QLora

/bin/bash: line 1: cd: /content/drive/MyDrive/: No such file or directory
Cloning into 'XLM-Finetune'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 292 (delta 23), reused 9 (delta 3), pack-reused 242
Receiving objects: 100% (292/292), 131.66 MiB | 13.47 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Updating files: 100% (224/224), done.


In [2]:
!pip install -r /content/XLM-Finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00


In [6]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('YOUR_TOKEN_HERE')"

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch # test torch

In [4]:
!python /content/XLM-Finetune/utils/squad_to_mrc.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Chunk /content/XLM-Finetune/data-bin/raw/squad/dev-context-en-question-vi.json: 100% 228/228 [00:02<00:00, 91.78it/s] 
511
Chunk /content/XLM-Finetune/data-bin/raw/squad/test-context-en-question-vi.json: 100% 2454/2454 [00:07<00:00, 334.83it/s]
5495
Chunk /content/XLM-Finetune/data-bin/raw/squad/test-context-vi-question-en.json: 100% 2454/2454 [00:03<00:00, 640.48it/s]
5495
Chunk /content/XLM-Finetune/data-bin/raw/squad/train_qa_vi_mailong.json: 100% 526/526 [00:00<00:00, 771.56it/s]
3454
Chunk /content/XLM-Finetune/data-bin/raw/squad/dev-context-vi-question-en.json: 100% 228/228 [00:00<00:00, 556.69it/s]
511
Total: 15466 samples


In [5]:
!python /content/XLM-Finetune/utils/train_valid_split.py

Train: 13919 samples
Valid: 1547 samples


In [17]:
from transformers import TrainingArguments, BitsAndBytesConfig, AutoTokenizer, Trainer
from XLMFinetune.utils import data_loader
from XLMFinetune.model.mrc_modelQLora import MRCQuestionAnswering
import transformers
import numpy as np
from datasets import load_metric
import os
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


#os.environ["CUDA_VISIBLE_DEVICES"] = ""

if __name__ == "__main__":
    model_id = "xlm-roberta-large"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quaQLorant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          cache_dir='/content/XLMFinetune/model-bin2/cache',
                                          #local_files_only=True
                                         )

    model = MRCQuestionAnswering.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0},cache_dir='/content/XLMFinetune/model-bin2/cache')
    print(model)
    #model.to("cuda")

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)

    # tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

    for i in range(24):
      config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=[
            f'roberta.encoder.layer.{i}.attention.self.query',
            f'roberta.encoder.layer.{i}.attention.self.key',
            f'roberta.encoder.layer.{i}.attention.self.value',
            f'roberta.encoder.layer.{i}.output.self.dense',
        ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
      )



    model = get_peft_model(model, config)


    train_dataset, valid_dataset = data_loader.get_dataloader(
        train_path='/content/XLMFinetune/data-bin/processed/train.dataset',
        valid_path='/content/XLMFinetune/data-bin/processed/valid.dataset'
    )

    #train_dataset = train_dataset.to("cuda")
    #valid_dataset = valid_dataset.to("cuda")

    '''
    train_dataset, valid_dataset = data_loader.get_dataloader(
        train_path='/content/XLMFinetune/data-bin/processed/train.dataset',
        valid_path='/content/XLMFinetune/data-bin/processed/valid.dataset'
    )
    '''

    training_args = TrainingArguments("/content/XLMFinetune/model-bin2/test",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=2,
                                      learning_rate=2e-4,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      fp16 = True,
                                      per_device_train_batch_size=1,
                                      per_device_eval_batch_size=1,
                                      gradient_accumulation_steps=1,
                                      logging_dir='./log',
                                      logging_steps=5,
                                      label_names=['start_positions',
                                                   'end_positions',
                                                   'span_answer_ids',
                                                   'input_ids',
                                                   'words_lengths'],
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      metric_for_best_model='f1',
                                      load_best_model_at_end=True,
                                      save_total_limit=2,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      evaluation_strategy="epoch",
                                      )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_loader.data_collator,
        compute_metrics=data_loader.compute_metrics,
        #data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )



    model.config.use_cache = False
    trainer.train()

    trainer.push_to_hub()


You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of MRCQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MRCQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (key): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (value): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear4bit(in_features=1024, out_features=1024, bias=Tr

Map (num_proc=10):   0%|          | 0/13919 [00:00<?, ? examples/s]

Filter (num_proc=10):   0%|          | 0/13919 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/1547 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=Tru

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/197k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

In [27]:
!python /content/XLMFinetune/infer.py

2024-01-16 06:31:36.520758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 06:31:36.520799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 06:31:36.521983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 06:31:37.638716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
model.safetensors: 100% 2.24G/2.24G [00:12<00:00, 177MB/s]
Some weights of MRCQues

In [28]:
# test model on huggin face

from transformers import pipeline
# model_checkpoint = "nguyenvulebinh/vi-mrc-large"
model_checkpoint = "Phanh2532/XLMQLoraCustom"
nlp = pipeline('question-answering', model=model_checkpoint,
                   tokenizer=model_checkpoint)
QA_input = {
  'question': "Một năm có bao nhiêu tháng có 31 ngày?",
  'context': "8 tháng"
}
res = nlp(QA_input)
print('pipeline: {}'.format(res))

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

pipeline: {'score': 0.12637759745121002, 'start': 2, 'end': 7, 'answer': 'tháng'}
